In [1]:
import os
# Find the latest version of spark 3.2 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.3'
spark_version = 'spark-3.2.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:2 http://archive.ubuntu.com/ubuntu focal InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease [18.1 kB]
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  Release
Get:7 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Get:8 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Get:10 http://security.ubuntu.com/ubuntu focal-security/main amd64 Packages [2,496 kB]
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:12 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Get:13 http://security.ubuntu.co

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2023-02-20 14:51:42--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  6.21MB/s    in 0.2s    

2023-02-20 14:51:43 (6.21 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M17-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [5]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_Games_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Video_Games_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   12039526| RTIS3L2M1F5SM|B001CXYMFS|     737716809|Thrustmaster T-Fl...|     Video Games|          5|            0|          0|   N|                Y|an amazing joysti...|Used this for Eli...| 2015-08-31|
|         US|    9636577| R1ZV7R40OLHKD|B00M920ND6|     569686175|Tonsee 6 buttons ...|     Video Games|          5|    

In [7]:
#Clean the DataFrame
cleaned_df = df.dropna()
cleaned_df.show()


+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   12039526| RTIS3L2M1F5SM|B001CXYMFS|     737716809|Thrustmaster T-Fl...|     Video Games|          5|            0|          0|   N|                Y|an amazing joysti...|Used this for Eli...| 2015-08-31|
|         US|    9636577| R1ZV7R40OLHKD|B00M920ND6|     569686175|Tonsee 6 buttons ...|     Video Games|          5|    

In [8]:
#Create the vine_table Dataframe
vine_df = cleaned_df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RTIS3L2M1F5SM|          5|            0|          0|   N|                Y|
| R1ZV7R40OLHKD|          5|            0|          0|   N|                Y|
|R3BH071QLH8QMC|          1|            0|          1|   N|                Y|
|R127K9NTSXA2YH|          3|            0|          0|   N|                Y|
|R32ZWUXDJPW27Q|          4|            0|          0|   N|                Y|
|R3AQQ4YUKJWBA6|          1|            0|          0|   N|                Y|
|R2F0POU5K6F73F|          5|            0|          0|   N|                Y|
|R3VNR804HYSMR6|          5|            0|          0|   N|                Y|
| R3GZTM72WA2QH|          5|            0|          0|   N|                Y|
| RNQOY62705W1K|          4|            0|          0|   N|     

Deliverable 2: Determine Bias of Vine Reviews

In [9]:
#Step 1: Filter the data and create a new DatFrame or table to reieve 
#all the rows where the total_votes count is equal to or greater 20
from pyspark.sql.functions import col
total_votes_df = vine_df.filter(col("total_votes") >= 20)
total_votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| R4PKAZRQJJX14|          1|           21|         34|   N|                N|
|R2CI0Y288CC7E2|          1|           21|         35|   N|                Y|
|R127WEQY2FM1T3|          1|          147|        175|   N|                Y|
|R3EZ0EPYLDA34S|          1|           14|         31|   N|                Y|
|R2FJ94555FZH32|          2|           55|         60|   N|                N|
|R1U3AR67RE273L|          1|           51|         65|   N|                Y|
|R3PZOXA5X1U8KW|          4|           31|         36|   N|                N|
| R6KTC1OPIOIIG|          2|           19|         34|   N|                Y|
|R36O341WWXXKNP|          5|           28|         31|   N|                N|
|R3GSK9MM8DNOYI|          1|            4|         32|   N|     

In [11]:
#Step 2: Retrieve all the rows where the number of helpful_votes divided by total_votes is equal or greater than 50%.
percent_votes_df = total_votes_df.withColumn('percent_votes', col('helpful_votes')
/col('total_votes')).alias('percent_votes').filter(col('percent_votes') >=.05)
percent_votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+--------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|       percent_votes|
+--------------+-----------+-------------+-----------+----+-----------------+--------------------+
| R4PKAZRQJJX14|          1|           21|         34|   N|                N|  0.6176470588235294|
|R2CI0Y288CC7E2|          1|           21|         35|   N|                Y|                 0.6|
|R127WEQY2FM1T3|          1|          147|        175|   N|                Y|                0.84|
|R3EZ0EPYLDA34S|          1|           14|         31|   N|                Y| 0.45161290322580644|
|R2FJ94555FZH32|          2|           55|         60|   N|                N|  0.9166666666666666|
|R1U3AR67RE273L|          1|           51|         65|   N|                Y|  0.7846153846153846|
|R3PZOXA5X1U8KW|          4|           31|         36|   N|                N|  0.8611111111111112|
| R6KTC1OP

In [12]:
#Step 3: Retrieve all rows where a review was written as part of the paid Vine program
paid_df = percent_votes_df.filter(col("vine") == "Y"). show()

+--------------+-----------+-------------+-----------+----+-----------------+-------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|      percent_votes|
+--------------+-----------+-------------+-----------+----+-----------------+-------------------+
|R3KKUSGFZWSUIY|          5|           56|         63|   Y|                N| 0.8888888888888888|
|R10FO5UKKVZBK2|          3|           23|         23|   Y|                N|                1.0|
| RM4KSGEOR7MU1|          5|           19|         24|   Y|                N| 0.7916666666666666|
|R18YQ4NRL2373C|          3|            2|         28|   Y|                N|0.07142857142857142|
| RG7VRMYLEXD23|          4|           22|         26|   Y|                N| 0.8461538461538461|
|R11O4YSCPSNL6L|          3|           20|         26|   Y|                N| 0.7692307692307693|
|R286MFBAJ8NPD6|          5|           46|         51|   Y|                N| 0.9019607843137255|
|R1JRR530H4COA2|    

In [13]:
#Step 4: Retrieve all rows where the review was not part of the paid Vine program
non_paid_df = percent_votes_df.filter(col("vine") == "N").show()

+--------------+-----------+-------------+-----------+----+-----------------+--------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|       percent_votes|
+--------------+-----------+-------------+-----------+----+-----------------+--------------------+
| R4PKAZRQJJX14|          1|           21|         34|   N|                N|  0.6176470588235294|
|R2CI0Y288CC7E2|          1|           21|         35|   N|                Y|                 0.6|
|R127WEQY2FM1T3|          1|          147|        175|   N|                Y|                0.84|
|R3EZ0EPYLDA34S|          1|           14|         31|   N|                Y| 0.45161290322580644|
|R2FJ94555FZH32|          2|           55|         60|   N|                N|  0.9166666666666666|
|R1U3AR67RE273L|          1|           51|         65|   N|                Y|  0.7846153846153846|
|R3PZOXA5X1U8KW|          4|           31|         36|   N|                N|  0.8611111111111112|
| R6KTC1OP

In [14]:
#Step 5: Determine the total number of reviews, number of 5 -star reviews, 
#and percentage of 5 -star reviews for the 2 types of reviews paid/unpaid

from pyspark.sql.functions import col,when,count,lit
ratings_total_df = percent_votes_df.groupBy("vine").agg(
    count(col("vine")).alias("Total_Reviews"),
    count(when(col("star_rating") == 5, True)).alias("Total_5_Star_Reviews"),
    (count(when(col("star_rating") ==5, True))/count(col("vine"))*100).alias("%_5_Star_To_Total")).show()


+----+-------------+--------------------+-----------------+
|vine|Total_Reviews|Total_5_Star_Reviews|%_5_Star_To_Total|
+----+-------------+--------------------+-----------------+
|   Y|          102|                  48|47.05882352941176|
|   N|        61173|               19922|32.56665522370981|
+----+-------------+--------------------+-----------------+

